# 前処理入門

- datasetは[こちら](https://github.com/ghmagazine/awesomebook)を参照

In [1]:
!python3 -V

Python 3.7.4


## Import

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import gc
import sys

setup

In [3]:
def memory_checker():
    print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
    print(" ------------------------------------ ")
    for var_name in globals():
        if not var_name.startswith("_"):
            print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

## example dataの読み込み関数

In [4]:
def reserve_data():
    URL = "https://raw.githubusercontent.com/ghmagazine/awesomebook/master/data/reserve.csv "
    return pd.read_csv(URL)

## 二つのpadas.DataFrameが等価かの確認

DataFrameのelementのdtypeが全く同じであるかの確認が前提

### NAを含むDataFrameを作成

In [5]:
np.random.seed(0)
df = pd.DataFrame(np.random.randint(1, 4, size=(3, 4)), columns=list("abcd"))
df["c"] = np.nan
other = df.copy()
df

,a,b,c,d
0,1,2,NaN,2
1,2,3,NaN,3
2,1,1,NaN,3


NaNはequalとはならない

In [6]:
df == other

,a,b,c,d
0,True,True,False,True
1,True,True,False,True
2,True,True,False,True


In [7]:
np.nan == np.nan, np.nan != np.nan

(False, True)

In [8]:
df.equals(other)

True

In [9]:
df = df.fillna('NA')
other = other.fillna('NA')
df

,a,b,c,d
0,1,2,NA,2
1,2,3,NA,3
2,1,1,NA,3


In [10]:
equal_df = df == other
equal_df

,a,b,c,d
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True


In [11]:
with pd.option_context("display.float_format", "{:.2f}%".format):
    print(pd.DataFrame(equal_df.sum(axis = 0)/equal_df.shape[0]*100).T)

        a       b       c       d
0 100.00% 100.00% 100.00% 100.00%


## [1]: データ列の抽出

https://raw.githubusercontent.com/ghmagazine/awesomebook/master/data/reserve.csv の予約テーブルデータから、`reserve_id`, `hotel_id`, `customer_id`, `reserve_datetime`だけに絞り込んだDataFrameを出力せよ

### 解答1 using loc

In [12]:
%%timeit
URL = "https://raw.githubusercontent.com/ghmagazine/awesomebook/master/data/reserve.csv "
df = pd.read_csv(URL)
df = df.loc[:, ['reserve_id', 'hotel_id', 'customer_id', 'reserve_datetime']]

262 ms ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


解答2 using usecols

In [13]:
%%timeit
URL = "https://raw.githubusercontent.com/ghmagazine/awesomebook/master/data/reserve.csv "
df = pd.read_csv(URL, usecols = ['reserve_id', 'hotel_id', 'customer_id', 'reserve_datetime'])

The slowest run took 4.45 times longer than the fastest. This could mean that an intermediate result is being cached.
300 ms ± 220 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 解答3 using sqlite3

In [14]:
%%timeit
URL = "https://raw.githubusercontent.com/ghmagazine/awesomebook/master/data/reserve.csv "
df = pd.read_csv(URL, usecols = ['reserve_id', 'hotel_id', 'customer_id', 'reserve_datetime'])

con = sqlite3.connect(':memory:')
df.to_sql('data', con)
res = pd.read_sql_query("""
SELECT 
    reserve_id, 
    hotel_id, 
    customer_id, 
    reserve_datetime
FROM 
    data  
""", con)

223 ms ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## [2] 条件によるデータ抽出

予約テーブルデータから`checkin_date`が2016-10-12から2016-10-13までのデータの行を抽出する。

In [15]:
df = reserve_data()

### 解1 using loc

In [16]:
%%timeit
df.loc[(df['checkin_date'] >= '2016-10-12') & (df['checkin_date'] <= '2016-10-13')]

2.3 ms ± 375 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 解2 using query

In [17]:
%%timeit
df.query('"2016-10-12" <= checkin_date <= "2016-10-13"')

2.81 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 解3 using index lambda

In [18]:
%%timeit
df.loc[lambda df: (df['checkin_date'] >= '2016-10-12') & (df['checkin_date'] <= '2016-10-13'), :]

1.95 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## [3] ランダムサンプリング

予約テーブルデータから50% randome sampling w.r.t rowで抽出する

In [19]:
df = reserve_data()

### 解答1: DataFrame.sample

In [20]:
%%timeit
df.sample(frac= 0.5, replace= False, random_state=1)

842 µs ± 7.78 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


An upsample sample of the DataFrame with replacement: Note that replace parameter has to be True for frac parameter > 1.

In [21]:
df

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100
...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000


In [22]:
df.sample(frac= 2, replace= True, random_state=1)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
1061,r1062,h_166,c_260,2016-06-07 02:57:45,2016-06-24,10:30:00,2016-06-27,4,199200
235,r236,h_111,c_54,2016-04-02 18:54:09,2016-04-24,11:00:00,2016-04-25,1,43600
3980,r3981,h_235,c_988,2016-11-09 15:11:02,2016-11-23,11:00:00,2016-11-26,4,50400
1096,r1097,h_147,c_267,2016-07-26 03:41:27,2016-07-31,12:00:00,2016-08-03,3,120600
3839,r3840,h_110,c_956,2018-01-31 11:05:49,2018-02-11,11:00:00,2018-02-13,3,158400
...,...,...,...,...,...,...,...,...,...
2439,r2440,h_44,c_609,2018-03-06 22:18:18,2018-03-08,12:00:00,2018-03-09,1,4900
3540,r3541,h_178,c_880,2017-02-24 05:52:38,2017-03-17,11:00:00,2017-03-18,2,42000
2967,r2968,h_212,c_749,2016-04-29 06:25:35,2016-05-19,09:30:00,2016-05-22,4,147600
2636,r2637,h_221,c_663,2016-03-04 13:44:01,2016-03-18,11:30:00,2016-03-19,2,11000


## [4] IDごとのランダムサンプリング

customer_id単位のランダムサンプリングを予約テーブルデータを用いて実行する

In [23]:
df = reserve_data()

### 解答1 DataFrame.unique

In [24]:
%%timeit
target = pd.Series(df['customer_id'].unique()).sample(frac = 0.5 ,replace= False, random_state=1)
df.loc[df['customer_id'].isin(target),:]

2.09 ms ± 90.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## [5] カウントとユニークカウント

予約テーブルから、ホテルごとに予約件数と予約した顧客数を算出せよ

### 解答1: using DataFrame.groupby

In [25]:
res = df.groupby('hotel_id').agg({'reserve_id':'count', 'customer_id':'nunique'})
res.reset_index(inplace = True)
res

,hotel_id,reserve_id,customer_id
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13
...,...,...,...
295,h_95,13,13
296,h_96,13,13
297,h_97,16,16
298,h_98,17,16


### 解答2 sqlite3

In [26]:
con = sqlite3.connect(':memory:')
df.to_sql('data', con)
res = pd.read_sql_query("""
SELECT
    hotel_id, 
    COUNT(DISTINCT reserve_id) AS reserved_cnt,
    COUNT(DISTINCT customer_id) AS customer_cnt
FROM 
    data  
GROUP BY
    1
""", con)
res

,hotel_id,reserved_cnt,customer_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13
...,...,...,...
295,h_95,13,13
296,h_96,13,13
297,h_97,16,16
298,h_98,17,16


## [6] 合計値

予約テーブルから、ホテルごとの宿泊人数別合計予約金額を算出する

### 解答1: using DataFrame.groupby

In [27]:
df.groupby(['hotel_id', 'people_num'])['total_price'].agg([np.sum]).rename(columns = {'sum': 'price_sum'})

price_sum
hotel_id people_num           
h_1      1              156600
         2              156600
         3              391500
         4              417600
h_10     1               11200
...                        ...
h_98     3              793800
         4              453600
h_99     1              179200
         2              448000
         4             1164800

[1159 rows x 1 columns]

### 解答2: Sqlite3

In [28]:
con = sqlite3.connect(':memory:')
df.to_sql('data', con)
res = pd.read_sql_query("""
SELECT
    hotel_id,
    people_num,
    SUM(total_price) AS price_sum
FROM 
    data  
GROUP BY
    1, 2
""", con)
res

,hotel_id,people_num,price_sum
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200
...,...,...,...
1154,h_98,3,793800
1155,h_98,4,453600
1156,h_99,1,179200
1157,h_99,2,448000


## [7] 代表値
予約テーブルから、ホテルごとの予約金額の最大値、最小値、平均値、中央値、20パーセントタイル値を算出せよ

### 解答1 : using DataFrame.describe()

In [29]:
df.groupby('hotel_id')['total_price'].describe(percentiles = [.20])

,count,mean,std,min,20%,50%,max
hotel_id,,,,,,,
h_1,10.0,112230.000000,56449.526127,26100.0,73080.0,104400.0,208800.0
h_10,3.0,42933.333333,28736.968061,11200.0,26880.0,50400.0,67200.0
h_100,20.0,27600.000000,17883.835689,4800.0,9600.0,28800.0,57600.0
h_101,17.0,75764.705882,49014.703676,14000.0,30800.0,56000.0,168000.0
h_102,13.0,32769.230769,18912.755159,12000.0,18000.0,24000.0,72000.0
...,...,...,...,...,...,...,...
h_95,13.0,275815.384615,182037.696857,43200.0,146880.0,259200.0,518400.0
h_96,13.0,33015.384615,17774.225072,7400.0,17760.0,29600.0,66600.0
h_97,16.0,83600.000000,73991.116584,20900.0,20900.0,62700.0,250800.0


### 解答2: using groupby and agg and quantile

In [30]:
res = df.groupby('hotel_id').agg({'total_price': ['max', 'min', 'mean',
                                            'median', lambda x: x.quantile(0.20)]}).reset_index()
res.columns = ['hotel_id', 'max', 
               'min', 'mean',
               'median', 'price_20percentile']
res

,hotel_id,max,min,mean,median,price_20percentile
0,h_1,208800,26100,112230.000000,104400,73080
1,h_10,67200,11200,42933.333333,50400,26880
2,h_100,57600,4800,27600.000000,28800,9600
3,h_101,168000,14000,75764.705882,56000,30800
4,h_102,72000,12000,32769.230769,24000,18000
...,...,...,...,...,...,...
295,h_95,518400,43200,275815.384615,259200,146880
296,h_96,66600,7400,33015.384615,29600,17760
297,h_97,250800,20900,83600.000000,62700,20900
298,h_98,226800,18900,96723.529412,75600,56700


### 解答3: np.percentile

In [31]:
res = df.groupby('hotel_id').agg({'total_price': ['max', 'min', 'mean',
                                            'median', lambda x: np.percentile(x, q = 20)]}).reset_index()
res.columns = ['hotel_id', 'max', 
               'min', 'mean',
               'median', 'price_20percentile']
res

,hotel_id,max,min,mean,median,price_20percentile
0,h_1,208800,26100,112230.000000,104400,73080
1,h_10,67200,11200,42933.333333,50400,26880
2,h_100,57600,4800,27600.000000,28800,9600
3,h_101,168000,14000,75764.705882,56000,30800
4,h_102,72000,12000,32769.230769,24000,18000
...,...,...,...,...,...,...
295,h_95,518400,43200,275815.384615,259200,146880
296,h_96,66600,7400,33015.384615,29600,17760
297,h_97,250800,20900,83600.000000,62700,20900
298,h_98,226800,18900,96723.529412,75600,56700


## [8] 分散値と標準偏差

予約テーブルから各ホテルの予約金額の分散値と標準偏差を算出せよ。計算できない時は0とすること

In [32]:
res = df.groupby('hotel_id').agg({'total_price': ['std', 'var']}).reset_index()
res.columns = ['hotel_id', 'std', 'var']
res.fillna(0, inplace = True)
res

,hotel_id,std,var
0,h_1,56449.526127,3.186549e+09
1,h_10,28736.968061,8.258133e+08
2,h_100,17883.835689,3.198316e+08
3,h_101,49014.703676,2.402441e+09
4,h_102,18912.755159,3.576923e+08
...,...,...,...
295,h_95,182037.696857,3.313772e+10
296,h_96,17774.225072,3.159231e+08
297,h_97,73991.116584,5.474685e+09
298,h_98,58590.896578,3.432893e+09


## [9] 最頻値の計算

予約テーブルの予約金額を1000単位にカテゴリ化して最頻値を算出せよ

### 解答1: using SQL

In [33]:
con = sqlite3.connect(':memory:')
df.to_sql('data', con)
res = pd.read_sql_query("""
SELECT
    ROUND(CAST(total_price AS FLOAT)/1000)*1000 AS total_price_round,
    COUNT(*)
FROM 
    data  
GROUP BY
    1
ORDER BY
    2 DESC
""", con)
res

,total_price_round,COUNT(*)
0,20000.0,58
1,50000.0,56
2,40000.0,55
3,10000.0,52
4,17000.0,49
...,...,...
371,776000.0,1
372,799000.0,1
373,822000.0,1
374,841000.0,1


### 間違った解答

Pythonのroundは浮動小数点の関係上, 10の位に四捨五入で丸める時、25は20へ、25.1は30が出力される

In [34]:
df['total_price'].round(-3).mode()

0    10000
1    20000
2    40000
dtype: int64

### 解答2

In [35]:
tmp = df['total_price'].apply(lambda x : x + 1e-7).round(-3)
tmp.value_counts()

20000.0     58
50000.0     56
40000.0     55
10000.0     52
17000.0     49
            ..
316000.0     1
388000.0     1
229000.0     1
205000.0     1
776000.0     1
Name: total_price, Length: 376, dtype: int64

## [10] 順位の算出

予約レコードを用いて、顧客ごとに予約日時の順位を古い順番につける。同じ予約日時の場合は、データの読み込み順に小さな順位をつける。

In [36]:
df = reserve_data()

### 解答1: SQL

In [37]:
%%timeit
query = """
SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY reserve_datetime) AS log_no
FROM 
    data
ORDER BY
    customer_id, reserve_datetime ASC
"""
con = sqlite3.connect(':memory:')
df.to_sql('data', con)
res = pd.read_sql_query(query, con)
res

41.5 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 解答2: using rank関数

In [38]:
%%timeit
df['reserve_datetime'] = pd.to_datetime(df['reserve_datetime'],
                                       format = '%Y-%m-%d %H:%M:%S')
df['log_no'] = df.groupby(['customer_id'])['reserve_datetime'].rank(ascending = True, method = 'first')
df.sort_values(['customer_id', 'reserve_datetime'])

8.64 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## [11] 集計値に対する順位付け

予約テーブルを利用して、ホテルごとの予約数に順位をつける。同じ予約数の場合は、同予約数の全ホテルに最小の順位をつける

In [39]:
df = reserve_data()

### 解答1: SQL

In [40]:
query = """
SELECT
    hotel_id,
    RANK() OVER (ORDER BY COUNT(DISTINCT reserve_id) DESC) AS rsv_cnt_rank
FROM
    data
GROUP BY
    1
ORDER BY 2
"""
con = sqlite3.connect(':memory:')
df.to_sql('data', con)
res = pd.read_sql_query(query, con)
res

,hotel_id,rsv_cnt_rank
0,h_241,1
1,h_144,2
2,h_142,3
3,h_37,3
4,h_178,5
...,...,...
295,h_76,293
296,h_129,297
297,h_140,297
298,h_17,297


### 解答2: Python using rank

In [41]:
tmp = df.groupby('hotel_id').size().reset_index()
tmp.columns = ['hotel_id', 'rsv_cnt']

tmp['rsv_cnt_rank'] = tmp['rsv_cnt'].rank(ascending = False, method = 'min')
tmp.drop('rsv_cnt', axis = 1, inplace = True)
tmp = tmp.sort_values(['rsv_cnt_rank', 'hotel_id']).reset_index(drop = True)
tmp['rsv_cnt_rank'] = tmp['rsv_cnt_rank'].astype(int)
tmp

,hotel_id,rsv_cnt_rank
0,h_241,1
1,h_144,2
2,h_142,3
3,h_37,3
4,h_178,5
...,...,...
295,h_76,293
296,h_129,297
297,h_140,297
298,h_17,297


二つが等しいか確認する

In [42]:
tmp.equals(res)

True

## [12] マスターテーブルの結合

予約テーブルとホテルテーブルを結合して、粛発人数が1人のビジネスホテルの予約レコードのみを取り出す

In [43]:
gc.collect()
reserve_url = "https://raw.githubusercontent.com/ghmagazine/awesomebook/master/data/reserve.csv "
hotel_url = "https://raw.githubusercontent.com/ghmagazine/awesomebook/master/data/hotel.csv "

reserve_df = pd.read_csv(reserve_url)
hotel_df = pd.read_csv(hotel_url)
reserve_df = reserve_df.reset_index(drop = False)
reserve_df.columns = ('sort_index', 'reserve_id', 'hotel_id', 'customer_id', 'reserve_datetime',
       'checkin_date', 'checkin_time', 'checkout_date', 'people_num',
       'total_price')

In [44]:
reserve_df.head()

,sort_index,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [45]:
hotel_df.head()

,hotel_id,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,h_1,26100,D,D-2,43.064569,141.511397,True
1,h_2,26400,A,A-1,35.715320,139.939446,True
2,h_3,41300,E,E-4,35.281572,136.988565,False
3,h_4,5200,C,C-3,38.431293,140.795615,False
4,h_5,13500,G,G-3,33.597291,130.533872,True


### 解答1: SQL

In [46]:
query = """
SELECT
    res.sort_index,
    res.reserve_id,
    res.hotel_id,
    res.customer_id,
    res.reserve_datetime,
    res.checkin_date,
    res.checkin_time,
    res.checkout_date,
    res.people_num,
    res.total_price,
    hotel.base_price,
    hotel.big_area_name,
    hotel.small_area_name,
    hotel.hotel_latitude,
    hotel.hotel_longitude,
    hotel.is_business
FROM
    res_df AS res
    INNER JOIN hotel_df AS hotel
        ON res.hotel_id = hotel.hotel_id
WHERE
    hotel.is_business = True
AND res.people_num = 1
ORDER BY res.sort_index
"""
con = sqlite3.connect(':memory:')
reserve_df.to_sql('res_df', con)
hotel_df.to_sql('hotel_df', con)
res = pd.read_sql_query(query, con)
res.head()

,sort_index,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,6,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,1
1,10,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.595248,130.633567,1
2,12,r13,h_223,c_2,2017-10-19 03:03:30,2017-10-21,09:30:00,2017-10-23,1,137000,68500,C,C-2,38.329097,140.698165,1
3,17,r18,h_132,c_3,2016-10-22 02:18:48,2016-11-12,12:00:00,2016-11-13,1,20400,20400,C,C-1,38.231842,140.797268,1
4,24,r25,h_277,c_4,2016-03-28 07:17:34,2016-04-07,10:30:00,2016-04-10,1,39300,13100,C,C-1,38.233985,140.795603,1


### 解答2: Python

In [47]:
py_df = pd.merge(reserve_df.query('people_num == 1'),
        hotel_df.query('is_business == True'),
        on = 'hotel_id',
        how = 'inner')
py_df.head()

,sort_index,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,6,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,True
1,996,r997,h_256,c_244,2016-10-15 22:47:40,2016-10-31,10:30:00,2016-11-02,1,69000,34500,C,C-1,38.237294,140.696131,True
2,2601,r2602,h_256,c_650,2016-05-10 00:42:56,2016-05-12,11:00:00,2016-05-14,1,69000,34500,C,C-1,38.237294,140.696131,True
3,3737,r3738,h_256,c_930,2017-04-12 09:53:00,2017-05-08,11:30:00,2017-05-09,1,34500,34500,C,C-1,38.237294,140.696131,True
4,10,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.595248,130.633567,True


DataFrameの等価性を確認

In [48]:
py_df.equals(res)

False

In [49]:
tmp_1 = py_df.sort_values(['sort_index'])
tmp_2 = res.sort_values(['sort_index'])

tmp_1 = tmp_1.reset_index(drop = True)
tmp_2 = tmp_2.reset_index(drop = True)

tmp_1.equals(tmp_2)

False

In [50]:
equal_df = tmp_1 == tmp_2
with pd.option_context("display.float_format", "{:.2f}%".format):
    print(pd.DataFrame(equal_df.sum(axis = 0)/equal_df.shape[0]*100).T)

   sort_index  reserve_id  hotel_id  customer_id  reserve_datetime  \
0     100.00%     100.00%   100.00%      100.00%           100.00%   

   checkin_date  checkin_time  checkout_date  people_num  total_price  \
0       100.00%       100.00%        100.00%     100.00%      100.00%   

   base_price  big_area_name  small_area_name  hotel_latitude  \
0     100.00%        100.00%          100.00%         100.00%   

   hotel_longitude  is_business  
0          100.00%      100.00%  


原因は`is_business`がbool(python)とint64(sql)でdtypeが異なっていたため

## [13] 条件別結合

ホテルごとに他のホテルをレコメンドしたい場合を考える。レコメンドするホテルは同じグループに属する任意の他のホテルとする。グループの作成方法は、ホテルテーブルの全てのホテルに対して、small_area_nameのホテルが20件以上ある場合は、small_area_name, それ以外の場合はbig_area_nameとしてhotel_idに対してグループを作成する。そのグループに基づき、hotel_idに対してレコメンドするホテルのhotel_idを紐付けDataFrameとして出力する

In [51]:
hotel_url = "https://raw.githubusercontent.com/ghmagazine/awesomebook/master/data/hotel.csv "
hotel_df = pd.read_csv(hotel_url)

### 解答1: SQL

In [52]:
query = """
WITH hotel_group AS(
    SELECT
        small_area_name,
        CASE 
            WHEN COUNT(DISTINCT hotel_id) >= 20 
            THEN small_area_name
            ELSE big_area_name
        END AS group_area
    FROM
        hotel_df
    GROUP BY
        1
)
SELECT DISTINCT
    master.hotel_id,
    rec.hotel_id AS rec_hotel_id
FROM
    hotel_df AS master
    INNER JOIN hotel_group AS g1
        ON master.small_area_name = g1.small_area_name
    INNER JOIN hotel_group AS g2
        ON g1.group_area = g2.group_area
    INNER JOIN hotel_df AS rec
        ON g2.small_area_name = rec.small_area_name
WHERE
    master.hotel_id <> rec.hotel_id
ORDER BY 
    1, 2
"""
con = sqlite3.connect(':memory:')
hotel_df.to_sql('hotel_df', con)
res = pd.read_sql_query(query, con)
res

,hotel_id,rec_hotel_id
0,h_1,h_103
1,h_1,h_109
2,h_1,h_119
3,h_1,h_124
4,h_1,h_125
...,...,...
9359,h_99,h_60
9360,h_99,h_65
9361,h_99,h_7
9362,h_99,h_93


### 解答2 : Python

In [53]:
tmp_df = hotel_df.groupby(['big_area_name', 'small_area_name'], as_index = False).size().reset_index()
tmp_df.columns = ['big_area_name', 'small_area_name', 'size']
tmp_df['group_name'] = tmp_df.apply(lambda x: x['small_area_name'] if x['size'] >= 20 else x['big_area_name'], axis = 1)
group_df = pd.merge(hotel_df, tmp_df.loc[:, ['small_area_name', 'group_name']], on = 'small_area_name', how = 'inner')
res_df = pd.merge(group_df, group_df.loc[:, ['group_name', 'hotel_id']], 
                  on = 'group_name', how = 'left', suffixes=('_master', '_rec'))
res_df = res_df.query('hotel_id_master != hotel_id_rec')
res_df = res_df.loc[:, ['hotel_id_master', 'hotel_id_rec']].sort_values(['hotel_id_master', 'hotel_id_rec']).reset_index(drop = True)
res_df

,hotel_id_master,hotel_id_rec
0,h_1,h_103
1,h_1,h_109
2,h_1,h_119
3,h_1,h_124
4,h_1,h_125
...,...,...
9359,h_99,h_60
9360,h_99,h_65
9361,h_99,h_7
9362,h_99,h_93


## [14] n件前のデータ取得

予約レコードを用いて、全ての行に同じ顧客の２回前の予約金額の情報を付与する。二回前の情報が存在しないrowに対してはNoneをreturnする。

In [54]:
del df
df = reserve_data()
df.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


### 解答1: SQL

In [55]:
query = """
SELECT
    t1.*,
    LAG(total_price, 2) OVER (PARTITION BY customer_id ORDER BY reserve_datetime) AS before_price
FROM
    data AS t1
"""
con = sqlite3.connect(':memory:')
df.to_sql('data', con)
res = pd.read_sql_query(query, con)
res = res.drop(['index'], axis = 1)
res.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,before_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,NaN
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,97200.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,20600.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,33600.0


### 解答２: Python using merge 

In [56]:
df['reserve_datetime'] = pd.to_datetime(df['reserve_datetime'],
                                       format = '%Y-%m-%d %H:%M:%S')
df['log_no'] = df.groupby(['customer_id'])['reserve_datetime'].rank(ascending = True, method = 'first')
df['log_no_2'] = df['log_no'] + 2
py_res = pd.merge(df,df.loc[:, ['customer_id', 'log_no_2', 'total_price']], 
         left_on = ['customer_id', 'log_no'],
         right_on = ['customer_id', 'log_no_2'],
         how = 'left',
         suffixes=('', '_right'))

py_res = py_res.drop(['log_no', 'log_no_2', 'log_no_2_right'], axis = 1)
py_res = py_res.rename(columns = {'total_price_right':'before_total_price'})
py_res.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,before_total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,NaN
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,97200.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,20600.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,33600.0


### 解答3: python using shift

In [57]:
shift_res = df.groupby(['customer_id']).apply(lambda x: x.sort_values(by = 'reserve_datetime'
                                                         , axis = 0,
                                                         inplace = False))
shift_res['before_price'] = shift_res['total_price'].shift(periods = 2) 
shift_res = shift_res.reset_index(drop = True)
shift_res.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no,log_no_2,before_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1.0,3.0,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2.0,4.0,NaN
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3.0,5.0,97200.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4.0,6.0,20600.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5.0,7.0,33600.0


## [15] 過去n件の合計値

予約レコードの全ての行に、地震から二件前までの３回の合計予約値を付与する。

In [58]:
del df
df = reserve_data()
df.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


### 解答1: SQL

In [59]:
query = """
SELECT
    t1.*,
    SUM(total_price) OVER (PARTITION BY customer_id ORDER BY reserve_datetime 
                           ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS price_sum
FROM
    data AS t1
"""
con = sqlite3.connect(':memory:')
df.to_sql('data', con)
res = pd.read_sql_query(query, con)
res = res.drop(['index'], axis = 1)
res.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,117800
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,151400
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,248600
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,296100


### 解答2 : Python

In [60]:
df['price_sum'] = pd.Series(df.groupby('customer_id').apply(lambda x: x.sort_values(by = 'reserve_datetime', ascending = True)
                               ).loc[:, 'total_price'].rolling(center = False, window = 3, min_periods = 0
                                                              ).sum().reset_index(drop = True))
df

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,97200.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,117800.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,151400.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,248600.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,296100.0
...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,217800.0
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,159000.0
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,112600.0
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,95000.0


## [16] 過去n件の平均値

予約テーブルの全ての行に、自身の行を含めないで１件前から３件前までの３回の平均予約金額の情報を付与する。

In [61]:
del df
df = reserve_data()
df.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


### 解答1 : SQL

In [62]:
query = """
SELECT
    *,
    AVG(total_price) OVER (PARTITION BY customer_id ORDER BY checkin_date ROWS 
                            BETWEEN 3 PRECEDING AND 1 PRECEDING) AS price_avg
FROM
    data
"""

con = sqlite3.connect(':memory:')
df.to_sql('data', con)
pd.read_sql_query(query, con)

,index,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_avg
0,0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,97200.000000
2,2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,58900.000000
3,3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,50466.666667
4,4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,82866.666667
...,...,...,...,...,...,...,...,...,...,...,...
4025,4023,r4024,h_244,c_999,2016-10-06 18:01:34,2016-10-07,11:30:00,2016-10-09,3,59400,62400.000000
4026,4024,r4025,h_160,c_999,2017-03-11 11:56:05,2017-03-27,10:00:00,2017-03-30,1,37200,60900.000000
4027,4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,53000.000000
4028,4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,37533.333333


### 解答2: Python rolling

In [63]:
df['price_avg'] = pd.Series(df.groupby('customer_id')
          .apply(lambda x: x.sort_values(by = 'checkin_date', ascending = True))
          .loc[:, 'total_price']
          .rolling(center = False, window = 3, min_periods = 1
            ).mean().reset_index(drop = True))

df['price_avg'] = df.groupby('customer_id')['price_avg'].shift(periods = 1)
df

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_avg
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,97200.000000
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,58900.000000
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,50466.666667
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,82866.666667
...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,139066.666667
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,72600.000000
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,53000.000000
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,NaN
